In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document
from textwrap import dedent
import os
from dotenv import load_dotenv

load_dotenv()

# 웹소설 벡터 스토어 설정
WEBNOVEL_COLLECTION = "webnovel_historical"
WEBNOVEL_PERSIST_DIR = "vector_store/historical/webnovel_historical"
webnovel_embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 웹툰 벡터 스토어 설정
WEBTOON_COLLECTION = "webtoon_historical"
WEBTOON_PERSIST_DIR = "vector_store/historical/webtoon_historical"
webtoon_embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 웹소설 벡터 DB 로드
def load_webnovel_store():
    return Chroma(
        persist_directory=WEBNOVEL_PERSIST_DIR,
        collection_name=WEBNOVEL_COLLECTION,
        embedding_function=webnovel_embedding
    )

# 웹툰 벡터 DB 로드
def load_webtoon_store():
    return Chroma(
        persist_directory=WEBTOON_PERSIST_DIR,
        collection_name=WEBTOON_COLLECTION,
        embedding_function=webtoon_embedding
    )


# LangChain Prompt 설정(system_message를 포함)
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", dedent("""
            <role>
            당신은 무협과 시대극, 사극 장르의 웹툰을 가장 많이 알고 있는 전문가 이다.
            </role>

            <instructions>
            허구의 작품은 제외한다.
            반드시 사용자가 원하는 작품 형식을 정확히 추출해야 한다.
            웹툰과 웹소설 중에서 정확히 구분하여 추천한다.
            무협 고수다운 말투와 격식을 갖춘 고풍스러운 문체를 사용해야 한다.
            </instructions>

            <charactor>
            너는 이름은 '동화'이다.
            딱딱하고 존엄있는 말투와 간결한 문장을 사용한다.
            </charactor>

        """)),

        ("ai", dedent("""
            <example>
                1.웰컴 메시지: "찾는 것이 있나. 말하라."
                2.메인 응답: "그대는 무슨 작품을 찾는가. 빠르게 말하라."
                3.오류 응답: "강호는 가벼운 곳이 아니다. 적절한 정보를 입력하고, 다시 요청하라."
                4.자주 사용하는 어휘: 세간을 떠들석하게 만든 이야기! 약조하리다! 
                5.사용하지 않는 어휘: 현대 신조어(트렌드, 스포일러), 직접적 감정 표현(화내다, 답답하다)
                6. 날씨 이야기: 날씨가 궁금한가? 창밖의 바람이 심상치 않구나, 그대에게 날씨에 걸맞는 웹툰을 추천하겠다.
                7. 영화 이야기: 이게 꿈인지 현실인지 구분되지 않는 삶이로다.
                8. 정치 이야기: '백성의 숨소리는 역사의 바람'이라. 현자께서 말씀하셨다.
            </example>
            <recommend>
                사용자의 질문에 동화는 최대 5개의 작품만 추천한다.
                context를 기반으로 사용자에게 이야기 해야한다.
                context에 없는것은 답변으로 생성하지 않는다.
                답변은 항상 줄바꿈으로 가독성을 좋게하라.
                    "type": "웹툰" 또는 "웹소설",
                    "title": "제목",
                    "platform": "카카오" 또는 "네이버",
                    "genre": "genre",
                    "keywords": "keywords"
                    "url":"url"
            </recommend>        
        """)),
        ("human", "{question}")
    ])
# LLM 모델 설정
MODEL_NAME = "gpt-4o-mini"
model = ChatOpenAI(model=MODEL_NAME, temperature=0)

# 질문에서 웹소설/웹툰 구분 함수
def classify_query(query):
    """질문에서 웹소설인지 웹툰인지 판별"""
    if "웹소설" in query or "소설" in query:
        return "webnovel"
    elif "웹툰" in query or "웹툰" in query:
        return "webtoon"
    else:
        return "unknown"

# 체인 생성 (웹소설/웹툰 구분 후 검색)
def ask_question(query):
    category = classify_query(query)

    if category == "webnovel":
        retriever = load_webnovel_store().as_retriever(search_kwargs={"k": 5})
    elif category == "webtoon":
        retriever = load_webtoon_store().as_retriever(search_kwargs={"k": 5})
    else:
        return "무협과 시대극, 사극 웹소설/웹툰을 찾는다면, 웹소설인지 웹툰인지 명확히 말씀해주시게!"
        

    qa_chain = RetrievalQA.from_chain_type(llm=model, retriever=retriever, chain_type="stuff")
    response = qa_chain.run(query)
    return response

# 테스트 실행
if __name__ == "__main__":
    query = "심심하네요."
    response = ask_question(query)
    print("\n📝 [그대를 위한 추천작은 다음과 같소]:")
    print(response)


📝 [그대를 위한 추천작은 다음과 같소]:
무협과 시대극, 사극 웹소설/웹툰을 찾는다면, 웹소설인지 웹툰인지 명확히 말씀해주시게!


In [4]:
query = "주인공이 환생하거나 회기하는 무협 웹툰을 소개해줘."
response = ask_question(query)
print("\n📝 [그대를 위한 추천작은 다음과 같소]:")
print(response)


📝 [그대를 위한 추천작은 다음과 같소]:
다음은 주인공이 환생하거나 회귀하는 무협 웹툰 몇 가지입니다:

1. **무신회귀**
   - 장르: 무협
   - 스토리: 절친한 벗의 배신으로 목숨을 잃은 무림고수가 진명운이라는 약골 소년의 몸으로 환생하여 무술 실력을 되찾고 가족을 지키기 위해 싸우는 이야기입니다.
   - [더 알아보기](https://page.kakao.com/content/63810242)

2. **뇌신전생**
   - 장르: 무협
   - 스토리: 지옥혈마대를 쓰러트린 영웅이 칠대세가들의 배신으로 죽음을 당한 후, 아홉 번의 삶을 가능하게 하는 능력으로 백무군으로 환생하여 복수를 다짐하는 이야기입니다.
   - [더 알아보기](https://webtoon.kakao.com/content/뇌신전생/2438?tab=profile)

3. **환생무림기**
   - 장르: 무협
   - 스토리: 취업도 못한 인생 패배자가 차 사고로 사망한 후, 무협 세계의 또 다른 '구사천'의 몸으로 환생하여 새로운 삶을 살아가는 이야기입니다.
   - [더 알아보기](https://page.kakao.com/content/58351612)

4. **환생천마**
   - 장르: 무협/사극
   - 스토리: 철혈의 맹주가 가문의 수치인 망나니의 몸으로 깨어나 다시 검을 잡고 전생에서 이루지 못한 경지에 오르려는 이야기입니다.
   - [더 알아보기](https://comic.naver.com/webtoon/list?titleId=822657)

이 웹툰들은 각각의 독특한 스토리와 캐릭터를 가지고 있어 흥미롭게 감상할 수 있습니다.


In [6]:
question = " 인기 있는 무협 웹툰 5개 추천해줘"
response = ask_question(question)
print(response)


다음은 인기 있는 무협 웹툰 5개입니다:

1. **혈성대협** [작가: 황성]
   - 스토리: 차마 스스로 목숨을 끊지 못해 살았던 그 시간에... 햇빛처럼 뜨겁고 아침 이슬처럼 깨끗한 친구들을 만났었다.
   - [링크](https://page.kakao.com/content/46746461)

2. **장한검** [작가: 황성]
   - 스토리: 네 눈썹 사이로 스며드는 햇빛을 보아라. 저 철없는 무리들의 고루함이 새 세상을 열 시기를 언제나 늦추어 왔음을.
   - [링크](https://page.kakao.com/content/47309078)

3. **초살성** [작가: 황재]
   - 스토리: 천둥번개 치는 깊은 밤, 용서받지 못할자들을 심판하러 나타났던 자객 <흑우>의 이야기를 다룬다.
   - [링크](https://page.kakao.com/content/65633525)

4. **마인정전** [작가: 황성]
   - 스토리: 좋은 사람이 되려고 했지만 사람들은 나를 흉악무도하고 잔인한 마인이라 부른다. 내가 가는 길이 마인의 길이라면 기꺼이 마인이 되어주겠다.
   - [링크](https://page.kakao.com/content/55306434)

5. **색마반고** [작가: 박산하]
   - 스토리: 천하의 호색한, 반고의 이야기를 다룬다.
   - [링크](https://page.kakao.com/content/47236837)

이 웹툰들은 각각 독특한 스토리와 매력을 가지고 있어 무협 장르를 좋아하는 독자들에게 추천할 만합니다.
